<h1>Public Attribute - Wo ist das Problem?</h1>

Attribute einer Klasse können als <code>public</code> deklariert werden, und sind es in Python auch, wenn deer Name nicht mit einem Underscore ('_') beginnt. Das macht deren Verwendung zwar sehr einfach, hat aber auch ein paar Probleme.

Schauen wir uns folgende Klassendefinition an:

In [151]:
from datetime import date

class Person:
    
    def __init__(self, name, ahvNummer, gebDatum):
        self.name = name
        self.ahv_nummer = ahvNummer
        self.geburtsdatum = date.fromisoformat(gebDatum) # Will raise an exception if not a valid datestring in isoformat

Damit können wir Objekte dieser Klasse instanziieren und die Attribute direkt manipulieren:

In [152]:
me = Person("Peter Müller", "756.1309.1284.04", "1975-01-13")

me.name = 567
me.ahv_nummer = "678"
me.geburtsdatum = 123

print("{}, geboren am {}, hat AHV-Nummer {}".format(me.name, me.geburtsdatum, me.ahv_nummer))

567, geboren am 123, hat AHV-Nummer 678


Wir sehen zwei Probleme:
* Die AHV-Nummer kann von aussen geändert werden. Die AHV-Nummer einer Person ändert sich aber nie; deshalb sollten Änderungen auch nicht erlaubt sein.
* Da Python nicht streng typisiert ist, kann ich dem Geburtsdatum auch unsinnige Werte, hier eine Zahl, zuweisen. Oder auch statt einem Namen eine Zahl.
* Ausserdem würden wir gerne das Alter der Person bekommen können, nicht einfach das Geburtsdatum.

Das Problem wird typischerweise über sog. Getter- und Setter-Funktionen gelöst. In Python gibt es für deren Implementierung 3 Varianten, wie unten beschrieben.

<h2>1. Normale Get- und Set-Methoden</h2>

Die Attribute werden <code>private</code> deklariert. Um den Wert eines Attributes zu lesen, wird dazu eine get-Methode definiert, in der wir auch Werte validieren können. Für die Attribute, die auch geschrieben werden dürfen, wird ausserdem eine set-Methode definiert:

In [163]:
from datetime import date
import re

class Person2:
    
    @staticmethod
    def __validate_name(name):
        # Validiere, ob es sich um einen gültigen Namen handelt.
        # Hier der Einfachheit halber nur Test, ob es sich um einen String ohne Sonderzeichen handelt,
        # sprich Buchstaben und Leerschläge
        if isinstance(name, str):
            regexp = re.compile('[^a-zA-Z äüöÄÜÖ]+')
            return (regexp.search(name) is None)
        else:
            return False
    
    def __init__(self, name, ahvNummer, gebDatum):
        if Person2.__validate_name(name):
            self.__name = name
        self.__ahv_nummer = ahvNummer #AHV number should be validated...
        self.__geburtsdatum = date.fromisoformat(gebDatum) # Will raise an exception if not a valid datestring in isoformat
        
    def get_name(self,):
        return self.__name
    
    def get_ahv_nummer(self,):
        return self.__ahv_nummer
    
    def get_geburtsdatum(self,):
        return self.__geburtsdatum
    
    def get_age(self,):
        #Berechne Alter
        today = date.today()
        age = today.year - self.__geburtsdatum.year - ((today.month, today.day) < (self.__geburtsdatum.month, self.__geburtsdatum.day))
        return age

    def set_name(self, name):
        if Person2.__validate_name(name): 
            self.__name = name
        else:
            raise ValueError("Kein gültiger Name!")
    
    

Jetzt können wir so ein Objekt instanzieren und auf die Attribute via getter- und setter-Methoden zugreifen:

In [164]:
me2 = Person2("Peter Müller", "756.1309.1284.04", "1975-01-13")

print("{}, geboren am {}, hat AHV-Nummer {}. Alter: {}".format(me2.get_name(), 
                                                               me2.get_geburtsdatum(), 
                                                               me2.get_ahv_nummer(), 
                                                               me2.get_age()))

Peter Müller, geboren am 1975-01-13, hat AHV-Nummer 756.1309.1284.04. Alter: 47


Ein falscher Name führt zu einem Fehler:

In [165]:
me2.set_name(123)

ValueError: Kein gültiger Name!

Die getter- und setter-Methoden sind ganz normale Methoden und könnten auch anders heissen. Es ist nur eine Namenskonvention, dass diese <code>get_<i>attributname</i></code> bzw. <code>set_<i>attributname</i></code> heissen.

<h2>2. Verwenden der <code>property()</code>-Funktion</h2>

Die <code>property()</code>-Funktion ist eine eingebaute Funktion, welche ein Property-Objekt erzeugt. Bei der Erzeugung werden die relevanten getter- und setter-Methoden mitgegeben, sowie optional auch eine Methode zum Löschen des Attributes, sowie eine Beschreibung. Bei read-only Attributen wird nur die entsprechende getter-Methode mitgegeben.

Klassendefinition könnte so aussehen:

In [168]:
from datetime import date
import re

class Person3:
    
    @staticmethod
    def __validate_name(name):
        # Validiere, ob es sich um einen gültigen Namen handelt.
        # Hier der Einfachheit halber nur Test, ob es sich um einen String ohne Sonderzeichen handelt,
        # sprich Buchstaben und Leerschläge
        if isinstance(name, str):
            regexp = re.compile('[^a-zA-Z äüöÄÜÖ]+')
            return (regexp.search(name) is None)
        else:
            return False
    
    def __init__(self, name, ahvNummer, gebDatum):
        if Person3.__validate_name(name):
            self.__name = name
        self.__ahv_nummer = ahvNummer #AHV number should be validated...
        self.__geburtsdatum = date.fromisoformat(gebDatum) # Will raise an exception if not a valid datestring in isoformat
        
    def _get_name(self,):
        return self.__name
    
    def _get_ahv_nummer(self,):
        return self.__ahv_nummer
    
    def _get_geburtsdatum(self,):
        return self.__geburtsdatum
    
    def _get_age(self,):
        #Berechne Alter
        today = date.today()
        age = today.year - self.__geburtsdatum.year - ((today.month, today.day) < (self.__geburtsdatum.month, self.__geburtsdatum.day))
        return age

    def _set_name(self, name):
        if Person3.__validate_name(name): 
            self.__name = name
        else:
            raise ValueError("Kein gültiger Name!")
    
    #Define properties using property function
    name = property(_get_name, _set_name)
    ahv_nummer = property(_get_ahv_nummer)  #read-only, keine setter-Methode
    geburtsdatum = property(_get_geburtsdatum)
    alter = property(_get_age)

In [169]:
me3 = Person3("Peter Müller", "756.1309.1284.04", "1975-01-13")

print("{}, geboren am {}, hat AHV-Nummer {}. Alter: {}".format(me3.name, 
                                                               me3.geburtsdatum, 
                                                               me3.ahv_nummer, 
                                                               me3.alter))

Peter Müller, geboren am 1975-01-13, hat AHV-Nummer 756.1309.1284.04. Alter: 47


Read-only Attribute können nicht geändert werden:


In [170]:
me3.alter=20


AttributeError: can't set attribute

<h2>3. Verwenden des <code>@property</code>-Decorators</h2>

Mit Hilfe von Decorators können Methoden direkt als getter- bzw. setter-Methoden definiert werden, ohne explizit die property()-Funktion zu verwenden.

Klassendefinition könnte so aussehen:

In [171]:
from datetime import date
import re

class Person4:
    
    @staticmethod
    def __validate_name(name):
        # Validiere, ob es sich um einen gültigen Namen handelt.
        # Hier der Einfachheit halber nur Test, ob es sich um einen String ohne Sonderzeichen handelt,
        # sprich Buchstaben und Leerschläge
        if isinstance(name, str):
            regexp = re.compile('[^a-zA-Z äüöÄÜÖ]+')
            return (regexp.search(name) is None)
        else:
            return False
    
    def __init__(self, name, ahvNummer, gebDatum):
        if Person4.__validate_name(name):
            self.__name = name
        self.__ahv_nummer = ahvNummer #AHV number should be validated...
        self.__geburtsdatum = date.fromisoformat(gebDatum) # Will raise an exception if not a valid datestring in isoformat
    
    @property
    def name(self,):
        return self.__name
    
    @property
    def ahv_nummer(self,):
        return self.__ahv_nummer
    
    @property
    def geburtsdatum(self,):
        return self.__geburtsdatum
    
    @property
    def age(self,):
        #Berechne Alter
        today = date.today()
        age = today.year - self.__geburtsdatum.year - ((today.month, today.day) < (self.__geburtsdatum.month, self.__geburtsdatum.day))
        return age

    @name.setter
    def name(self, name):
        if Person4.__validate_name(name): 
            self.__name = name
        else:
            raise ValueError("Kein gültiger Name!")

In [172]:
me4 = Person4("Peter Müller", "756.1309.1284.04", "1975-01-13")

print("{}, geboren am {}, hat AHV-Nummer {}. Alter: {}".format(me4.name, 
                                                               me4.geburtsdatum, 
                                                               me4.ahv_nummer, 
                                                               me4.age))

Peter Müller, geboren am 1975-01-13, hat AHV-Nummer 756.1309.1284.04. Alter: 47


Read-only Attribute können nicht geändert werden, andere schon (inkl. Validierung!):

In [173]:
me4.age=20

AttributeError: can't set attribute

In [174]:
me4.name = "Sascha Meyer"
me4.name

'Sascha Meyer'